In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import haversine_distances
from math import radians
from fuzzywuzzy import fuzz
import itertools
from scipy.spatial import cKDTree
import numpy as np

gives me a dataframe that check all teh duplicate station based on the distance threshold

In [ ]:
def load_and_preprocess_data(file_path):
    """Load the Excel file and preprocess the data."""
    # Load data
    df = pd.read_excel(file_path)
    
    # Clean and standardize latitude/longitude
    # Convert to numeric, handling any errors
    df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
    df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')
    
    # Drop rows with missing lat/long
    df_clean = df.dropna(subset=['latitude', 'longitude'])
    
    # Create standardized text fields for matching
    # Handle missing values by filling with empty strings instead of 'nan'
    df_clean['gauge_name_clean'] = df_clean['Gauge Name/Number'].fillna('').astype(str).str.lower().str.strip()
    df_clean['station_name_clean'] = df_clean['Station_name'].fillna('').astype(str).str.lower().str.strip()
    df_clean['river_name_clean'] = df_clean['River Name (if reported)'].fillna('').astype(str).str.lower().str.strip()
    
    # Flag rows with valid metadata for smarter matching later
    df_clean['has_gauge_name'] = ((df_clean['gauge_name_clean'] != '') & (df_clean['gauge_name_clean'] != 'nan'))
    df_clean['has_station_name'] = ((df_clean['station_name_clean'] != '') & (df_clean['station_name_clean'] != 'nan'))
    df_clean['has_river_name'] = ((df_clean['river_name_clean'] != '') & (df_clean['river_name_clean'] != 'nan'))
    
    print(f"Original rows: {len(df)}, After cleaning: {len(df_clean)}")
    return df_clean

def find_potential_duplicates(df, distance_threshold_km=0.2):
    """Find potential duplicate stations based on geographic proximity."""
    # Extract sources and create pairs of sources
    sources = df['Name of Providing Entity'].unique()
    source_pairs = list(itertools.combinations(sources, 2))
    
    # Initialize results container
    potential_duplicates = []
    
    # For each pair of sources
    for source1, source2 in source_pairs:
        df1 = df[df['Name of Providing Entity'] == source1]
        df2 = df[df['Name of Providing Entity'] == source2]
        
        print(f"Comparing {len(df1)} stations from {source1} with {len(df2)} stations from {source2}")
        
        # Convert threshold to radians (haversine_distances expects radians)
        threshold_radians = distance_threshold_km / 6371.0  # Earth radius in km
        
        # For each station in the first source
        for idx1, row1 in df1.iterrows():
            # Convert coordinates to radians
            lat1, lon1 = radians(row1['latitude']), radians(row1['longitude'])
            
            # For each station in the second source
            for idx2, row2 in df2.iterrows():
                # Convert coordinates to radians
                lat2, lon2 = radians(row2['latitude']), radians(row2['longitude'])
                
                # Calculate haversine distance
                distance = haversine_distances([[lat1, lon1]], [[lat2, lon2]])[0][0] * 6371.0  # Convert to km
                
                # If stations are close enough
                if distance <= distance_threshold_km:
                    # Calculate name similarity scores with handling for missing data
                    gauge_name_sim = fuzz.ratio(row1['gauge_name_clean'], row2['gauge_name_clean']) if (row1['has_gauge_name'] and row2['has_gauge_name']) else 0
                    station_name_sim = fuzz.ratio(row1['station_name_clean'], row2['station_name_clean']) if (row1['has_station_name'] and row2['has_station_name']) else 0
                    river_name_sim = fuzz.ratio(row1['river_name_clean'], row2['river_name_clean']) if (row1['has_river_name'] and row2['has_river_name']) else 0
                    
                    # Count how many name fields are available for comparison
                    available_comparisons = (
                        int(row1['has_gauge_name'] and row2['has_gauge_name']) + 
                        int(row1['has_station_name'] and row2['has_station_name']) + 
                        int(row1['has_river_name'] and row2['has_river_name'])
                    )
                    
                    # Create a match record
                    match = {
                        'idx1': idx1,
                        'idx2': idx2,
                        'source1': source1,
                        'source2': source2,
                        'gauge_name1': row1['Gauge Name/Number'],
                        'gauge_name2': row2['Gauge Name/Number'],
                        'station_name1': row1['Station_name'],
                        'station_name2': row2['Station_name'],
                        'river_name1': row1['River Name (if reported)'],
                        'river_name2': row2['River Name (if reported)'],
                        'country1': row1['Country'],
                        'country2': row2['Country'],
                        'lat1': row1['latitude'],
                        'lon1': row1['longitude'],
                        'lat2': row2['latitude'],
                        'lon2': row2['longitude'],
                        'distance_km': distance,
                        'gauge_name_similarity': gauge_name_sim,
                        'station_name_similarity': station_name_sim,
                        'river_name_similarity': river_name_sim
                    }
                    potential_duplicates.append(match)
    
    # Convert to DataFrame
    results_df = pd.DataFrame(potential_duplicates)
    
    # Calculate overall similarity score with adaptive weights based on available data
    if not results_df.empty:
        # Add count of available comparisons
        results_df['available_comparisons'] = results_df.apply(
            lambda x: ((x['gauge_name_similarity'] > 0) + 
                      (x['station_name_similarity'] > 0) + 
                      (x['river_name_similarity'] > 0)), 
            axis=1
        )
        
        # Calculate normalized weights based on what's available
        results_df['overall_similarity'] = results_df.apply(
            lambda x: (
                (x['gauge_name_similarity'] * (0.3 if x['gauge_name_similarity'] > 0 else 0)) +
                (x['station_name_similarity'] * (0.3 if x['station_name_similarity'] > 0 else 0)) +
                (x['river_name_similarity'] * (0.2 if x['river_name_similarity'] > 0 else 0)) +
                ((1 - x['distance_km'] / distance_threshold_km) * (0.2 + (0.8 - (x['available_comparisons'] * 0.2))))
            ) / (0.2 + (x['available_comparisons'] * 0.2)),  # Normalize by the weights used
            axis=1
        )
        
        # Sort by overall similarity
        results_df = results_df.sort_values('overall_similarity', ascending=False)
    
    return results_df

def classify_duplicates(duplicate_df, high_threshold=80, medium_threshold=60):
    """Classify potential duplicates into high, medium, and low confidence."""
    if duplicate_df.empty:
        return pd.DataFrame()
    
    # Add confidence classification
    duplicate_df['confidence'] = 'Low'
    duplicate_df.loc[duplicate_df['overall_similarity'] >= medium_threshold, 'confidence'] = 'Medium'
    duplicate_df.loc[duplicate_df['overall_similarity'] >= high_threshold, 'confidence'] = 'High'
    
    return duplicate_df

def optimize_for_large_dataset(df, distance_threshold_km=0.2):
    """Optimized approach for very large datasets using geospatial partitioning."""
    
    
    # Extract sources
    sources = df['Name of Providing Entity'].unique()
    source_pairs = list(itertools.combinations(sources, 2))
    
    potential_duplicates = []
    
    for source1, source2 in source_pairs:
        df1 = df[df['Name of Providing Entity'] == source1]
        df2 = df[df['Name of Providing Entity'] == source2]
        
        print(f"Comparing {len(df1)} stations from {source1} with {len(df2)} stations from {source2}")
        
        # Create KD-Tree for faster spatial search
        # Convert lat/lon to radians for proper distance calculation
        coords2_rad = np.radians(df2[['latitude', 'longitude']].values)
        tree = cKDTree(coords2_rad)
        
        # Threshold in radians
        threshold_rad = distance_threshold_km / 6371.0
        
        # For each station in the first source
        for idx1, row1 in df1.iterrows():
            # Skip if coordinates are missing
            if pd.isna(row1['latitude']) or pd.isna(row1['longitude']):
                continue
                
            # Convert to radians
            lat1_rad, lon1_rad = radians(row1['latitude']), radians(row1['longitude'])
            
            # Query the KD-Tree for neighbors within threshold
            indices = tree.query_ball_point([lat1_rad, lon1_rad], threshold_rad)
            
            # Process matches
            for i in indices:
                idx2 = df2.iloc[i].name
                row2 = df2.iloc[i]
                
                # Skip if countries are different
                if row1['Country'] != row2['Country']:
                    continue
                
                # Calculate actual distance
                lat2_rad, lon2_rad = radians(row2['latitude']), radians(row2['longitude'])
                distance = haversine_distances([[lat1_rad, lon1_rad]], [[lat2_rad, lon2_rad]])[0][0] * 6371.0
                
                # Calculate similarity scores with handling for missing data
                gauge_name_sim = fuzz.ratio(row1['gauge_name_clean'], row2['gauge_name_clean']) if (row1['has_gauge_name'] and row2['has_gauge_name']) else 0
                station_name_sim = fuzz.ratio(row1['station_name_clean'], row2['station_name_clean']) if (row1['has_station_name'] and row2['has_station_name']) else 0
                river_name_sim = fuzz.ratio(row1['river_name_clean'], row2['river_name_clean']) if (row1['has_river_name'] and row2['has_river_name']) else 0
                
                # Count how many name fields are available for comparison
                available_comparisons = (
                    int(row1['has_gauge_name'] and row2['has_gauge_name']) + 
                    int(row1['has_station_name'] and row2['has_station_name']) + 
                    int(row1['has_river_name'] and row2['has_river_name'])
                )
                
                # Get byu_ids
                byu_id1 = row1.get('byu_id', None)
                byu_id2 = row2.get('byu_id', None)
                
                # Create match record
                match = {
                    'idx1': idx1,
                    'idx2': idx2,
                    'source1': source1,
                    'source2': source2,
                    'gauge_name1': row1['Gauge Name/Number'],
                    'gauge_name2': row2['Gauge Name/Number'],
                    'station_name1': row1['Station_name'],
                    'station_name2': row2['Station_name'],
                    'river_name1': row1['River Name (if reported)'],
                    'river_name2': row2['River Name (if reported)'],
                    'country1': row1['Country'],
                    'country2': row2['Country'],
                    'lat1': row1['latitude'],
                    'lon1': row1['longitude'],
                    'lat2': row2['latitude'],
                    'lon2': row2['longitude'],
                    'distance_km': distance,
                    'gauge_name_similarity': gauge_name_sim,
                    'station_name_similarity': station_name_sim,
                    'river_name_similarity': river_name_sim,
                    'byu_id1': byu_id1,
                    'byu_id2': byu_id2,
                    'available_comparisons': available_comparisons
                }
                potential_duplicates.append(match)
    
    # Convert to DataFrame
    results_df = pd.DataFrame(potential_duplicates)
    
    # Calculate overall similarity score
    if not results_df.empty:
        results_df['overall_similarity'] = (
            results_df['gauge_name_similarity'] * 0.3 + 
            results_df['station_name_similarity'] * 0.3 + 
            results_df['river_name_similarity'] * 0.2 + 
            (1 - results_df['distance_km'] / distance_threshold_km) * 0.2
        )
        
        # Sort by overall similarity
        results_df = results_df.sort_values('overall_similarity', ascending=False)
    
    return results_df

def mark_duplicates_in_dataframe(original_df, duplicate_pairs_df, confidence_threshold=60):
    """
    Add 'duplicated' and 'primary_gauge' columns to the original dataframe.
    The 'primary_gauge' column will contain the byu_ids of the duplicated rows.
    """
    print("Marking duplicates in the original dataframe...")
    
    # Filter by confidence threshold
    filtered_pairs = duplicate_pairs_df[
        (duplicate_pairs_df['overall_similarity'] >= confidence_threshold)
    ]
    
    if filtered_pairs.empty:
        print("No duplicates found meeting the confidence threshold.")
        original_df['duplicated'] = False
        original_df['primary_gauge'] = ""
        return original_df
    
    print(f"Number of duplicate pairs after filtering: {len(filtered_pairs)}")
    
    # Create new columns in the original dataframe
    original_df['duplicated'] = False
    original_df['primary_gauge'] = ""
    
    # Dictionary to collect all duplicates for each row
    duplicate_mapping = {}
    
    # Track which indices are processed
    processed_indices = set()
    
    # Process each duplicate pair
    pair_count = 0
    for _, row in filtered_pairs.iterrows():
        pair_count += 1
        if pair_count % 1000 == 0:
            print(f"Processing pair {pair_count}/{len(filtered_pairs)}")
            
        idx1, idx2 = int(row['idx1']), int(row['idx2'])
        
        # Debugging info
        if pair_count <= 5:
            print(f"Pair {pair_count} indices: {idx1}, {idx2}")
            print(f"These indices exist in original_df: {idx1 in original_df.index}, {idx2 in original_df.index}")
        
        # Check if indices exist in the dataframe
        if idx1 not in original_df.index or idx2 not in original_df.index:
            print(f"Warning: Index {idx1} or {idx2} not found in original dataframe")
            continue
            
        # Mark both rows as duplicates
        original_df.loc[idx1, 'duplicated'] = True
        original_df.loc[idx2, 'duplicated'] = True
        
        processed_indices.add(idx1)
        processed_indices.add(idx2)
        
        # Get byu_ids and convert to strings
        byu_id1 = str(row['byu_id1']) if not pd.isna(row['byu_id1']) else ""
        byu_id2 = str(row['byu_id2']) if not pd.isna(row['byu_id2']) else ""
        
        # Add to duplicate mapping
        if idx1 not in duplicate_mapping:
            duplicate_mapping[idx1] = set()
        if idx2 not in duplicate_mapping:
            duplicate_mapping[idx2] = set()
            
        # Add the other's byu_id to each row's set
        if byu_id2:
            duplicate_mapping[idx1].add(byu_id2)
        if byu_id1:
            duplicate_mapping[idx2].add(byu_id1)
    
    # Fill the primary_gauge column with collected byu_ids
    for idx, byu_ids in duplicate_mapping.items():
        if byu_ids:  # Only set if there are actual byu_ids
            original_df.loc[idx, 'primary_gauge'] = ",".join(str(id) for id in byu_ids)
    
    # Count duplicates
    duplicate_count = original_df['duplicated'].sum()
    
    print(f"Found {duplicate_count} duplicate stations across {len(processed_indices)} unique indices")
    return original_df

def main(file_path, distance_threshold_km=0.2, use_optimized=True, confidence_threshold=0):
    """Main function to process the file and find duplicates."""
    # Load the original data without preprocessing
    print("Loading original data...")
    original_df = pd.read_excel(file_path)
    
    # Load and preprocess data for comparison
    print("Preprocessing data for comparison...")
    df = load_and_preprocess_data(file_path)
    
    # Find potential duplicates
    print("Finding potential duplicates...")
    if use_optimized and len(df) > 10000:
        print(f"Using optimized approach for large dataset with {len(df)} rows")
        duplicates_df = optimize_for_large_dataset(df, distance_threshold_km)
    else:
        duplicates_df = find_potential_duplicates(df, distance_threshold_km)
    
    print(f"Found {len(duplicates_df)} potential duplicate pairs before classification")
    
    # Classify duplicates
    classified_df = classify_duplicates(duplicates_df)
    print(f"After classification: {len(classified_df)} duplicate pairs")
    
    if not classified_df.empty:
        # Mark duplicates in the original dataframe
        result_df = mark_duplicates_in_dataframe(original_df, classified_df, confidence_threshold)
        
        # Save the updated original dataframe
        output_file = file_path.replace('.xlsx', '_with_duplicates_markedbgnbnnbmnbmnbmbnmchcjgcjgx.xlsx')
        result_df.to_excel(output_file, index=False)
        print(f"Updated dataframe saved to {output_file}")
        
        # Summary statistics
        print("\nDuplicate Summary:")
        print(f"Total duplicate pairs found: {len(classified_df)}")
        print(f"Total duplicate stations marked: {result_df['duplicated'].sum()}")
    else:
        print("No potential duplicates found.")
        # Still add the columns but mark everything as non-duplicate
        original_df['duplicated'] = False
        original_df['primary_gauge'] = ""
        
        output_file = file_path.replace('.xlsx', '_with_duplicates_marked.xlsx')
        original_df.to_excel(output_file, index=False)
    
    return original_df
# Usage
duplicate_df = main('/Users/yubinbaaniya/Downloads/combined_file_with_metadata jan9 with random identification_3.xlsx', distance_threshold_km=0.4)

based on above created dataframe it'll check which gaueg among the duplicated one is "the" main file

In [ ]:
def identify_main_gauges(file_path):
    """
    Identifies the main gauge between duplicates based on source priority.
    
    Parameters:
    file_path (str): Path to the Excel file with duplicates marked
    
    Returns:
    pd.DataFrame: DataFrame with added "main gauge" column
    """
    print("Loading Excel file...")
    df = pd.read_excel(file_path)
    
    # Create the main gauge column with default empty values
    df['main gauge'] = ""
    
    # Define source priorities (lower value = higher priority)
    source_priorities = {
        'CARAVAN': 2,  # Lowest priority
        'GRDC': 1,     # Second lowest priority
        # All other sources will get priority 0 (highest)
    }
    
    # Process each row with other_gauge values
    processed_count = 0
    for idx, row in df.iterrows():
        # If other_gauge is empty, skip
        if pd.isna(row['primary_gauge']) or row['primary_gauge'] == "":
            continue
        
        # Get the byu_id of the current row
        current_byu_id = row['byu_id']
        if pd.isna(current_byu_id):
            continue
            
        # Get the list of other gauge IDs
        other_gauge_ids = str(row['primary_gauge']).split(',')
        
        # For tracking the best priority among duplicates
        best_priority = float('inf')
        best_id = None
        
        # Get priority of the current row
        current_source = row['Name of Providing Entity']
        current_priority = source_priorities.get(current_source, 0)  # Default to 0 (highest) if not in dict
        
        # Track all related byu_ids (current one + all in other_gauge)
        all_related_ids = [str(current_byu_id)] + other_gauge_ids
        all_related_ids = [x.strip() for x in all_related_ids if x.strip()]
        
        # Record the best ID so far (which might be the current row)
        if current_priority < best_priority:
            best_priority = current_priority
            best_id = current_byu_id
        
        # Find rows for each other gauge ID and compare priorities
        for gauge_id in other_gauge_ids:
            if gauge_id.strip() == "":
                continue
                
            # Find rows with matching byu_id
            matching_rows = df[df['byu_id'].astype(str) == gauge_id.strip()]
            
            if not matching_rows.empty:
                # Get the first matching row (there should be only one)
                match_row = matching_rows.iloc[0]
                match_source = match_row['Name of Providing Entity']
                match_priority = source_priorities.get(match_source, 0)  # Default to 0 if not in dict
                
                # Check if this is a better priority
                if match_priority < best_priority:
                    best_priority = match_priority
                    best_id = match_row['byu_id']
        
        # Mark the main gauge
        if best_id is not None:
            # Find all rows with byu_id in the related IDs list
            for rel_id in all_related_ids:
                matching_rows = df[df['byu_id'].astype(str) == rel_id.strip()]
                if not matching_rows.empty:
                    for match_idx in matching_rows.index:
                        # Is this the row with the best priority?
                        if str(df.loc[match_idx, 'byu_id']) == str(best_id):
                            df.loc[match_idx, 'main gauge'] = "1"
        
        processed_count += 1
        if processed_count % 100 == 0:
            print(f"Processed {processed_count} rows with other_gauge values")
    
    # Save the updated file
    output_file = file_path.replace('.xlsx', '_with_main_gauge.xlsx')
    df.to_excel(output_file, index=False)
    print(f"Completed processing. Main gauge identified for {df['main gauge'].value_counts().get('1', 0)} rows.")
    print(f"Updated file saved to: {output_file}")
    
    return df

# Usage
df_result = identify_main_gauges('/Users/yubinbaaniya/Downloads/combined_file_with_metadata jan9 with random identification_3_with_duplicates_markedbgnbnnbmnbmnbmbnmchcjgcjgx.xlsx')